In [1]:
import synapseclient
from synapseclient import Project, File, Folder, Wiki
from synapseclient import Schema, Column, Table, Row, RowSet, as_table_columns
import itertools
import pandas as pd

In [2]:
syn = synapseclient.Synapse()
syn.login()

Welcome, Victor Baham!



### Create a Citations wiki subpage in the ADKP backend portal for each study where its name is the study name in the `Study` column of [Portal - Studies Table](https://www.synapse.org/#!Synapse:syn17083367/tables/)

#### Note: start at index 4 where index i = 0, 1, 2… n because I already manually created Citations wiki subpages for the first 4 studies

In [3]:
query_ps = syn.tableQuery(f"SELECT * FROM {syn.get('syn17083367').id}")
portal_studies = pd.read_csv(query_ps.filepath)

#### Create `Citations` column

In [4]:
portal_studies['Citations'] = ''

#### Note: The name of each Synapse entity in the `Study` column can be accessed with `syn.get('synID').name`

In [5]:
portal_studies

,ROW_ID,ROW_VERSION,Study_Type,Program,Study,DataType_All,Study_Name,Study_Description,specimenType,studyFocus,...,isModelSystem,Grant Number,Methods,Related_Studies,isFeatured,Acknowledgement,ackContext,studyMetadata,accessReqs,Citations
0,60,599,Consortium,"[""AMP-AD""]",syn5550419,"[""Gene Expression""]",The RNAseq Sample Swap Study (rnaSeqSampleSwap),This study provides transcriptomic data from a...,"[""Brain""]","[""No Cognitive Impairment""]",...,False,"[""cross-consortium"", ""U01AG046152"", ""U01AG0461...",syn20823962,"syn9702085, syn3219045, syn3159438, syn5550404",False,syn12666371/wiki/608162,syn12666371/wiki/617506,syn12666371/wiki/607136,syn12666371/wiki/595380,
1,61,599,Individual,"[""AMP-AD""]",syn16779040,"[""Gene Expression""]",The AD Cross Species Study (AD_CrossSpecies),This study provides transcriptomic data from a...,"[""Brain""]","[""Alzheimer Disease""]",...,True,"[""U01AG046161""]","syn20806503, syn17014486","syn6037590, syn10378810, syn9889979, syn598722...",False,syn12666371/wiki/607083,syn12666371/wiki/617506,syn12666371/wiki/607136,syn12666371/wiki/608104,
2,62,599,Individual,"[""AMP-AD""]",syn5759376,"[""Proteomics""]",The Adult Changes in Thought (ACT) Study,This study provides proteomic data from the d...,"[""Brain""]","[""Alzheimer Disease""]",...,False,"[""U01AG046161""]","syn17010684, syn20807073",syn16779040,False,syn12666371/wiki/602458,syn12666371/wiki/617506,syn12666371/wiki/607136,syn12666371/wiki/595380,
3,63,673,Individual,"[""AMP-AD"", ""M20VE-AD""]",syn9705278,"[""Metabolomics""]",The Alzheimer's Disease Metabolomics Consortiu...,This study provides metabolomics data generate...,"[""Blood""]","[""Alzheimer Disease""]",...,False,"[""RF1AG051550""]","syn9779065, syn9779089, syn52167479",syn5592519,False,syn12666371/wiki/602456,syn12666371/wiki/617506,syn12666371/wiki/608609,syn12666371/wiki/608105,
4,64,599,Individual,"[""AMP-AD""]",syn5592519,"[""Metabolomics""]",The Alzheimer's Disease Metabolomics Consortiu...,This study provides metabolomics data generate...,"[""Blood""]","[""Alzheimer Disease""]",...,False,"[""R01AG046171""]","syn12036813, syn9993525, syn7440346",syn9705278,False,syn12666371/wiki/602455,syn12666371/wiki/617506,syn12666371/wiki/608609,syn12666371/wiki/608105,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,272,669,Consortium,"[""MODEL-AD""]",syn50944316,"[""Gene Expression, Electrophysiology, Immunoas...",The UCI_Bin1K358R Study,"This study provides bulk RNA-seq, immunohistoc...","[""Brain""]","[""Alzheimer Disease"", "" Electrophysiology"", ""I...",...,True,"[""U54AG054349""]","syn51754040,syn51754034,syn51754035",NaN,False,syn12666371/wiki/620480,syn12666371/wiki/617506,syn12666371/wiki/607136,syn12666371/wiki/608104,
127,273,670,Consortium,NaN,syn27207345,NaN,The UCI_ABCA7 Study,NaN,"[""Brain""]","[""Alzheimer Disease""]",...,True,"[""U54AG054349""]",NaN,NaN,False,NaN,NaN,NaN,NaN,
128,274,672,Individual,"[""Psych-AD""]",syn52160016,"[""Gene Expression""]",The Mount Sinai Neuropsychiatric Symptoms in A...,This study provides single-nucleus RNAseq prof...,"[""Brain""]","[""Alzheimer Disease"", ""Parkinson Disease"", ""Fr...",...,False,"[""R01AG067025""]",syn57373409,"syn51732482, syn3159438,",False,syn12666371/wiki/627200,syn12666371/wiki/617506,syn12666371/wiki/607136,syn12666371/wiki/595380,
129,275,675,Individual,"[""M2OVE-AD""]",syn31511672,"[""Gene Expression""]",The Mayo Clinic Single Nucleus RNAseq Study (M...,This study provides single-nucleus transcripto...,"[""Brain""]","[""Alzheimer Disease""]",...,False,"[""RF1AG051504""]",syn57431217,NaN,False,syn12666371/wiki/627337,syn12666371/wiki/617506,syn12666371/wiki/607136,syn12666371/wiki/595380,


In [47]:
study_citations = [Wiki(title=syn.get(study).name, 
                        owner='syn12666371',
                        markdown=f"", 
                        parentWikiId='628124') 
                   for study in portal_studies['Study'] if
                  study not in list(portal_studies['Study'].iloc[range(0,4)])]

#### Use `sqlite3` as a temporary fix until it's decided to version the Portal - Studies Table [section under development]

In [48]:
import sqlite3

#### Double-check length of `study_citations` to ensure all 131 rows are captured

In [49]:
len(study_citations)

127

#### There are 127 wiki objects in the list, and 4 already created, so the correct number of 127 + 4 = 131 wikis are created

#### Use `syn.store` to finalize creating the Citations wikis

In [51]:
"""for sc in study_citations:
    syn.store(sc)"""

#### Get Citation wiki ids

In [48]:
import requests

response = requests.get('https://repo-prod.prod.sagebase.org/repo/v1/entity/syn12666371/wikiheadertree2')


if response.status_code == 200:
    data = response.json()
    
else:
    print(f'Unsuccessful: {response.status_code}')
    print(response.text)

In [50]:
print(data)

{'totalNumberOfResults': 51, 'results': [{'id': '576279', 'title': ''}, {'id': '581939', 'title': 'About', 'parentId': '576279'}, {'id': '617505', 'title': 'Acknowledgement Statement Context', 'parentId': '576279'}, {'id': '602387', 'title': 'Acknowledgment Statements', 'parentId': '576279'}, {'id': '595383', 'title': 'Agora', 'parentId': '576279'}, {'id': '625598', 'title': 'Analysis Platforms', 'parentId': '576279'}, {'id': '628124', 'title': 'Citations', 'parentId': '576279'}, {'id': '600032', 'title': 'Contribute Data', 'parentId': '576279'}, {'id': '581937', 'title': 'Data Access', 'parentId': '576279'}, {'id': '605050', 'title': 'Frequently Asked Questions', 'parentId': '576279'}, {'id': '607137', 'title': 'Homepage- Resources Module', 'parentId': '576279'}, {'id': '608500', 'title': 'Old Synapse Portal Content', 'parentId': '576279'}, {'id': '595379', 'title': 'Study Details', 'parentId': '576279'}, {'id': '582408', 'title': "What's New", 'parentId': '576279'}, {'id': '585318', 

In [51]:
import requests

response = requests.get('https://repo-prod.prod.sagebase.org/repo/v1/entity/syn628124/wikiheadertree2')


if response.status_code == 200:
    data = response.json()
    
else:
    print(f'Unsuccessful: {response.status_code}')
    print(response.text)

Unsuccessful: 404
{"concreteType":"org.sagebionetworks.repo.model.ErrorResponse","reason":"A root wiki does not exist for ownerId: 628124 and ownerType: ENTITY"}
